# Projeto Estágio supervisionado

Neste Projeto da cadeira "Estágio Supervisionado", vou trabalhar com um conjunto de dados relacionado ao petróleo. 

O conjunto de dados consiste em bases de dados que mostram sobre sua produção(1997 - 2025), processamento(1990 - 2025), importação(2000 - 2025), exportação(2000 - 2025) e, por ultimo, sobre o combustivel automotivo que tem total relação com o petróleo(2004, 2014, 2019, 2020, 2022 e 2025). Vamos alinhar todos os dados para o ano de 2004 para ficarem num mesmo ponto de partida.

O objetivo desde notbook é fazer uma análise dos dados completa para podermos seguir para os próximos passos são envolvendo Modelagem de dados, SQL, POWER BI (Dashboards) e Machine learning.

### Pipeline do Projeto de Análise de Dados de Petróleo e Combustíveis

#### Fase 1: Estruturação e Entendimento (Onde estamos)
1.1. Definição do Problema de Negócio
1.2. Coleta e Análise Exploratória PRELIMINAR
- Para cada arquivo CSV, faremos uma análise inicial (.info(), .head())
- Documentaremos os achados (tipos de dados a corrigir, colunas importantes)
1.3. Modelagem de Dados
- Com base nos achados, vamos desenhar nosso Star Schema (Tabelas Fato e Dimensão)

#### Fase 2: Engenharia de Dados (ETL - Extract, Transform, Load)
2.1. Criação da Estrutura no Banco de Dados
- Escreveremos o código SQL para criar as tabelas vazias no MySQL.
2.2. Desenvolvimento do Script de Carga e Tratamento
- Criaremos o script Python que lê os CSVs, limpa os dados (corrige tipos, trata nulos), e carrega nas tabelas do MySQL.

#### Fase 3: Visualização inicial com POWER BI
- 3.1. Conexão do Power BI ao Banco de Dados
- 3.2. Criação de dashboards iniciais 

#### Fase 4: Análise Exploratória de Dados PROFUNDA (EDA)
- 4.1. Análise Univariada (estudo de cada variável)
- 4.2. Análise Bivariada e Multivariada (cruzamento de variáveis)
- 4.3. Geração de Gráficos e Descoberta de Insights

#### Fase 5: Modelagem Preditiva (Machine Learning)
- 5.1. Preparação dos Dados para o Modelo
- 5.2. Treinamento de Modelos Não-Supervisionados (clusters, anomalias)
- 5.3. Interpretação dos Resultados

#### Fase 6: Visualização e Apresentação (Power BI)
- 6.2. Criação de Dashboards Interativos
- 6.3. Apresentação dos Resultados e Insights



## *Análise Inicial - Produção petroleo*

In [1]:
# Importando bibliotecas
import pandas as pd
import os

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'producao-petroleo-m3-1997-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [ ]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'producao-petroleo-m3-1997-2025.csv' carregado com sucesso!


In [ ]:
# Visualizando o DataFrame 5 primeiros
df.head()

,ANO,MÊS,GRANDE REGIÃO,UNIDADE DA FEDERAÇÃO,PRODUTO,LOCALIZAÇÃO,PRODUÇÃO
0,1997,JAN,REGIÃO NORTE,AMAZONAS,PETRÓLEO,TERRA,122200
1,1997,FEV,REGIÃO NORTE,AMAZONAS,PETRÓLEO,TERRA,124785
2,1997,MAR,REGIÃO NORTE,AMAZONAS,PETRÓLEO,TERRA,128177
3,1997,ABR,REGIÃO NORTE,AMAZONAS,PETRÓLEO,TERRA,124968
4,1997,MAI,REGIÃO NORTE,AMAZONAS,PETRÓLEO,TERRA,115710


In [ ]:
# Visualizando o DataFrame 5 ultimos
df.tail()

,ANO,MÊS,GRANDE REGIÃO,UNIDADE DA FEDERAÇÃO,PRODUTO,LOCALIZAÇÃO,PRODUÇÃO
7651,2025,AGO,REGIÃO SUL,PARANÁ,PETRÓLEO,MAR,0
7652,2025,SET,REGIÃO SUL,PARANÁ,PETRÓLEO,MAR,0
7653,2025,OUT,REGIÃO SUL,PARANÁ,PETRÓLEO,MAR,0
7654,2025,NOV,REGIÃO SUL,PARANÁ,PETRÓLEO,MAR,0
7655,2025,DEZ,REGIÃO SUL,PARANÁ,PETRÓLEO,MAR,0


In [ ]:
# Visualizando o DataFrame informações adicionais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656 entries, 0 to 7655
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ANO                   7656 non-null   int64 
 1   MÊS                   7656 non-null   object
 2   GRANDE REGIÃO         7656 non-null   object
 3   UNIDADE DA FEDERAÇÃO  7656 non-null   object
 4   PRODUTO               7656 non-null   object
 5   LOCALIZAÇÃO           7656 non-null   object
 6   PRODUÇÃO              7656 non-null   object
dtypes: int64(1), object(6)
memory usage: 418.8+ KB



### Análise Preliminar - `producao-petroleo-m3-1997-2025.csv`

#### Estrutura Geral
* O arquivo contém 7.656 registros e 7 colunas.
* A contagem de `non-null` indica que não há valores nulos (faltando) no conjunto de dados.

#### Colunas de Dimensão (Contexto)
* **`ANO`**: Lido corretamente como número (**`int64`**). Será usado na nossa dimensão de tempo.
* **`MÊS`**: Lido como texto (**`object`**). **Ação Necessária:** Converter para um formato numérico (1 para JAN, 2 para FEV, etc.) durante o ETL.
* **`GRANDE REGIÃO`**, **`UNIDADE DA FEDERAÇÃO`**: Lido como texto (**`object`**), o que é esperado. Serão colunas da nossa dimensão de localização.
* **`PRODUTO`**: Lido como texto (**`object`**). Será a base da nossa dimensão de produto.
* **`LOCALIZAÇÃO`**: Lido como texto (**`object`**, com valores 'TERRA' e 'MAR'). Pode se tornar uma coluna na dimensão de produto ou local.

#### Coluna Fato (A Métrica)
* **`PRODUÇÃO`**: Lido como texto (**`object`**). **Ação Crítica Necessária:** Principal métrica do arquivo que precisa ser convertida para um tipo numérico. A causa precisa ser investigada na Fase 2 (ETL).

#### Padronização
* Todos os nomes de colunas precisam ser convertidos para o formato `snake_case` (minúsculas, sem acentos, com `_`).

## *Análise Inicial - Processamento petroleo*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'processamento-petroleo-m3-1990-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [27]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'processamento-petroleo-m3-1990-2025.csv' carregado com sucesso!


In [ ]:
df.head()   

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,REFINARIA,MATÉRIA PRIMA,PROCESSADO
0,1990,DEZ,BAHIA,DAX OIL,OUTRAS CARGAS,0
1,1990,SET,BAHIA,DAX OIL,PETRÓLEO IMPORTADO,0
2,1990,OUT,BAHIA,DAX OIL,PETRÓLEO IMPORTADO,0
3,1990,NOV,BAHIA,DAX OIL,PETRÓLEO IMPORTADO,0
4,1990,DEZ,BAHIA,DAX OIL,PETRÓLEO IMPORTADO,0


In [29]:
df.tail()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,REFINARIA,MATÉRIA PRIMA,PROCESSADO
22666,2025,ABR,SÃO PAULO,SSOIL,PETRÓLEO NACIONAL,799
22667,2025,MAR,SÃO PAULO,SSOIL,PETRÓLEO NACIONAL,0
22668,2025,FEV,SÃO PAULO,SSOIL,PETRÓLEO NACIONAL,0
22669,2025,MAI,SÃO PAULO,RPBC,PETRÓLEO IMPORTADO,136
22670,2025,JUN,SÃO PAULO,SSOIL,OUTRAS CARGAS,0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22671 entries, 0 to 22670
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ANO                   22671 non-null  int64 
 1   MÊS                   22671 non-null  object
 2   UNIDADE DA FEDERAÇÃO  22671 non-null  object
 3   REFINARIA             22671 non-null  object
 4   MATÉRIA PRIMA         22671 non-null  object
 5   PROCESSADO            22671 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.0+ MB


### Análise Preliminar - `processamento-petroleo.csv`

#### Estrutura Geral
* O arquivo possui 22.671 registros e 6 colunas.
* Pela contagem de `non-null`, não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **`ANO`** e **`MÊS`**: Padrão já identificado. `ANO` está correto como `int64`, e `MÊS` é `object` (texto), precisando de tratamento futuro.
* **`UNIDADE DA FEDERAÇÃO`**: Coluna em comum com o arquivo de produção, servindo como ponto de conexão.
* **`REFINARIA`**: **Nova Pista!** Forte candidata a se tornar uma nova Tabela Dimensão (`d_refinaria`).
* **`MATÉRIA PRIMA`**: **Nova Pista!** Candidata a uma Tabela Dimensão ou a fazer parte da dimensão de produto (`d_produto`).

#### Coluna Fato (A Métrica)
* **`PROCESSADO`**: **Ótima Notícia!** Diferente do arquivo anterior, esta métrica já foi lida como um número (**`int64`**), indicando dados mais limpos.

#### Padronização
* Assim como no outro arquivo, todos os nomes de colunas precisarão ser padronizados para o formato `snake_case`.

## *Análise Inicial - importações e exportações petroleo*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'importacoes-exportacoes-petroleo-2000-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [32]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'importacoes-exportacoes-petroleo-2000-2025.csv' carregado com sucesso!


In [33]:
df.head()

,ANO,MÊS,PRODUTO,OPERAÇÃO COMERCIAL,IMPORTADO / EXPORTADO,DISPÊNDIO / RECEITA
0,2000,ABR,PETRÓLEO,EXPORTAÇÃO,0,0
1,2000,AGO,PETRÓLEO,EXPORTAÇÃO,0,0
2,2000,DEZ,PETRÓLEO,EXPORTAÇÃO,"419950,4202",59930153
3,2000,FEV,PETRÓLEO,EXPORTAÇÃO,"0,068537786",51
4,2000,JAN,PETRÓLEO,EXPORTAÇÃO,0,0


In [34]:
df.tail()

,ANO,MÊS,PRODUTO,OPERAÇÃO COMERCIAL,IMPORTADO / EXPORTADO,DISPÊNDIO / RECEITA
609,2025,JAN,PETRÓLEO,IMPORTAÇÃO,"1289574,654",633572724
610,2025,JUL,PETRÓLEO,IMPORTAÇÃO,"1154166,107",533184316
611,2025,JUN,PETRÓLEO,IMPORTAÇÃO,"1255895,879",547698306
612,2025,MAI,PETRÓLEO,IMPORTAÇÃO,"1173650,256",527925790
613,2025,MAR,PETRÓLEO,IMPORTAÇÃO,"1220041,255",602886781


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANO                    614 non-null    int64 
 1   MÊS                    614 non-null    object
 2   PRODUTO                614 non-null    object
 3   OPERAÇÃO COMERCIAL     614 non-null    object
 4   IMPORTADO / EXPORTADO  614 non-null    object
 5   DISPÊNDIO / RECEITA    614 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 28.9+ KB


### Análise Preliminar - `importacoes-exportacoes-petroleo.csv`

#### Estrutura Geral
* O arquivo é "leve", contendo 614 registros e 6 colunas, ideal para a nossa análise estrutural.
* A contagem de `non-null` indica que não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **`ANO`**, **`MÊS`**, **`PRODUTO`**: Dimensões que se repetem em outros arquivos, confirmando que serão nossas chaves de conexão principais. `MÊS` continua sendo `object` e precisará de tratamento.
* **`OPERAÇÃO COMERCIAL`**: **Pista Importantíssima!** Esta é uma nova dimensão categórica, com valores "IMPORTAÇÃO" e "EXPORTAÇÃO". Ela será fundamental para separar nossas métricas.

#### Colunas Fato (As Métricas)
* Este arquivo é rico em métricas, contendo duas colunas de fatos.
* **`IMPORTADO / EXPORTADO`**: Representa o volume/peso da operação. **Ação Crítica Necessária:** Foi lido como texto (**`object`**) mas claramente contém números. Precisa ser convertido para um tipo numérico (float/decimal) na fase de ETL.
* **`DISPÊNDIO / RECEITA`**: Representa o valor monetário da operação. **Boa Notícia:** Foi lido corretamente como um número (**`int64`**), indicando dados mais limpos nesta coluna.

#### Padronização
* Os nomes das colunas (ex: `IMPORTADO / EXPORTADO`) contêm espaços e `/`. Precisarão ser padronizados para `snake_case` (ex: `volume_kg`, `valor_usd`).

## *Análise Inicial - importações e exportações derivados*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'importacoes-exportacoes-derivados-2000-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [37]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'importacoes-exportacoes-derivados-2000-2025.csv' carregado com sucesso!


In [38]:
df.head()

,ANO,MÊS,PRODUTO,OPERAÇÃO COMERCIAL,IMPORTADO / EXPORTADO,DISPÊNDIO / RECEITA
0,2000,ABR,ASFALTO,EXPORTAÇÃO,"1362,035577",309443
1,2000,AGO,ASFALTO,EXPORTAÇÃO,"2700,793269",544917
2,2000,DEZ,ASFALTO,EXPORTAÇÃO,"1839,723077",426760
3,2000,FEV,ASFALTO,EXPORTAÇÃO,"1676,491346",270633
4,2000,JAN,ASFALTO,EXPORTAÇÃO,"1702,567308",360648


In [39]:
df.tail()

,ANO,MÊS,PRODUTO,OPERAÇÃO COMERCIAL,IMPORTADO / EXPORTADO,DISPÊNDIO / RECEITA
9187,2025,JAN,ÓLEO DIESEL,IMPORTAÇÃO,"1244998,07",722859013
9188,2025,JUL,ÓLEO DIESEL,IMPORTAÇÃO,"1725630,464",982821398
9189,2025,JUN,ÓLEO DIESEL,IMPORTAÇÃO,"1292890,907",683145669
9190,2025,MAI,ÓLEO DIESEL,IMPORTAÇÃO,"1459037,881",734590259
9191,2025,MAR,ÓLEO DIESEL,IMPORTAÇÃO,"1296733,98",708538917


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9192 entries, 0 to 9191
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANO                    9192 non-null   int64 
 1   MÊS                    9192 non-null   object
 2   PRODUTO                9192 non-null   object
 3   OPERAÇÃO COMERCIAL     9192 non-null   object
 4   IMPORTADO / EXPORTADO  9192 non-null   object
 5   DISPÊNDIO / RECEITA    9192 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 431.0+ KB


### Análise Preliminar - `importacoes-exportacoes-derivados.csv`

#### Estrutura Geral
* O arquivo contém 9.192 registros e 6 colunas.
* A contagem de `non-null` indica, novamente, que não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **Estrutura Idêntica:** As colunas de dimensão (`ANO`, `MÊS`, `PRODUTO`, `OPERAÇÃO COMERCIAL`) são **exatamente as mesmas** do arquivo de importação/exportação de petróleo cru. Isso é ótimo, pois significa que poderemos usar a mesma lógica de ETL para tratar ambos os arquivos.
* **`PRODUTO`**: Esta coluna irá enriquecer nossa dimensão de produto com derivados importantes, como 'ASFALTO' e 'ÓLEO DIESEL'.
* **`MÊS`**: Continua sendo `object`, confirmando a necessidade de tratamento.

#### Colunas Fato (As Métricas)
* **Padrão Confirmado:** As duas colunas de métricas seguem o mesmo padrão do arquivo anterior.
* **`IMPORTADO / EXPORTADO`**: **Ação Crítica Necessária:** Confirmado que esta coluna é lida como texto (**`object`**) e precisará ser convertida para um tipo numérico (float/decimal).
* **`DISPÊNDIO / RECEITA`**: Lido corretamente como número (**`int64`**). **Ação de Modelagem:** No Star Schema, esta coluna deverá ser modelada como `DECIMAL(18, 4)` para garantir a precisão financeira.

#### Padronização
* A padronização dos nomes das colunas para o formato `snake_case` será necessária, assim como nos outros arquivos.

## *Análise Inicial - combustivel automotivo 2025.01*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'ca-2019-02.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [3]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'ca-2019-02.csv' carregado com sucesso!


In [5]:
df.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GASOLINA,01/07/2019,"4,259",NaN,R$ / litro,BRANCA
1,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,ETANOL,01/07/2019,"4,099",NaN,R$ / litro,BRANCA
2,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GNV,01/07/2019,"3,449",NaN,R$ / m³,BRANCA
3,NE,BA,ITABUNA,LOPES LEMOS COMERCIO DE COMBUSTIVEIS LTDA,00.231.792/0001-05,RODOVIA BR 101,SN,KM 503 5,MANOEL LEAO,45601-402,GASOLINA,01/07/2019,"4,69",NaN,R$ / litro,BRANCA
4,NE,BA,ITABUNA,LOPES LEMOS COMERCIO DE COMBUSTIVEIS LTDA,00.231.792/0001-05,RODOVIA BR 101,SN,KM 503 5,MANOEL LEAO,45601-402,ETANOL,01/07/2019,"3,49",NaN,R$ / litro,BRANCA


In [6]:
df.tail()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
507294,SE,SP,LORENA,AUTO POSTO VIP LORENA EIRELI,28.039.566/0001-46,RUA JOAQUIM LUIS BASTOS,38,CENTRO,CRUZ,12606-090,DIESEL S10,30/12/2019,"3,799",NaN,R$ / litro,IPIRANGA
507295,SE,MG,SAO SEBASTIAO DO PARAISO,FC AUTO POSTO EIRELI,28.470.480/0001-73,AVENIDA ENGENHEIRO WASHINGTON MARTONI,1720,NaN,JARDIM PLANALTO,37950-000,GASOLINA,30/12/2019,"4,939","4,318",R$ / litro,BRANCA
507296,SE,MG,SAO SEBASTIAO DO PARAISO,FC AUTO POSTO EIRELI,28.470.480/0001-73,AVENIDA ENGENHEIRO WASHINGTON MARTONI,1720,NaN,JARDIM PLANALTO,37950-000,ETANOL,30/12/2019,"3,339","2,7697",R$ / litro,BRANCA
507297,SE,MG,SAO SEBASTIAO DO PARAISO,FC AUTO POSTO EIRELI,28.470.480/0001-73,AVENIDA ENGENHEIRO WASHINGTON MARTONI,1720,NaN,JARDIM PLANALTO,37950-000,DIESEL,30/12/2019,"3,799","3,416",R$ / litro,BRANCA
507298,SE,MG,SAO SEBASTIAO DO PARAISO,FC AUTO POSTO EIRELI,28.470.480/0001-73,AVENIDA ENGENHEIRO WASHINGTON MARTONI,1720,NaN,JARDIM PLANALTO,37950-000,DIESEL S10,30/12/2019,"3,899","3,47",R$ / litro,BRANCA


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507299 entries, 0 to 507298
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Regiao - Sigla     507299 non-null  object
 1   Estado - Sigla     507299 non-null  object
 2   Municipio          507299 non-null  object
 3   Revenda            507299 non-null  object
 4   CNPJ da Revenda    507299 non-null  object
 5   Nome da Rua        507299 non-null  object
 6   Numero Rua         507062 non-null  object
 7   Complemento        132458 non-null  object
 8   Bairro             505329 non-null  object
 9   Cep                507299 non-null  object
 10  Produto            507299 non-null  object
 11  Data da Coleta     507299 non-null  object
 12  Valor de Venda     507299 non-null  object
 13  Valor de Compra    190730 non-null  object
 14  Unidade de Medida  507299 non-null  object
 15  Bandeira           507299 non-null  object
dtypes: object(16)
memory

In [5]:
# Célula para contar a frequência de cada valor único
frequencia = df['Unidade de Medida'].value_counts()
print(frequencia)

Unidade de Medida
R$ / litro    499263
R$ / m³         8036
Name: count, dtype: int64


### Análise Preliminar - Arquivos de Preços de Combustíveis (Amostra)
*Análise baseada em amostras dos arquivos de 2019-02 e 2025-01.*

#### Estrutura Geral
* **Volume:** São arquivos grandes, com centenas de milhares de linhas e 16 colunas.
* **Qualidade dos Dados:** Identificamos três problemas de qualidade importantes:
    1.  **`DtypeWarning` (Aviso de Tipos Mistos):** O aviso no arquivo de 2025 indica que o Pandas encontrou inconsistências nos tipos de dados ao ler o arquivo. **Ação:** Nosso processo de ETL na Fase 2, onde forçaremos a conversão para os tipos corretos, resolverá este aviso.
    2.  **Linhas Vazias no Final:** O arquivo de 2025 apresenta várias linhas completamente vazias (`NaN`) no final do arquivo. **Ação:** Precisaremos adicionar uma etapa no ETL para remover essas linhas vazias.
    3.  **Valores Nulos (Missing Values):** Há uma grande quantidade de valores nulos, especialmente nas colunas `Complemento` e, de forma crítica, na `Valor de Compra`.

#### Colunas de Dimensão (Contexto)
* **Localização (Muito Detalhada):** Este arquivo enriquece enormemente nossa visão de localização, com `Regiao - Sigla`, `Estado - Sigla`, `Municipio`, `Bairro`, `Cep`, `Nome da Rua`. Estas colunas serão a base para uma `d_localidade` muito robusta.
* **Revendedor (Nova Dimensão Confirmada):** As colunas `Revenda`, `CNPJ da Revenda` e `Bandeira` confirmam a necessidade de criarmos uma Tabela Dimensão `d_revendedor`.
* **Produto e Tempo:**
    * `Produto`: Confirma a conexão com nossa `d_produto`.
    * `Data da Coleta`: Lido como texto (**`object`**). **Ação Crítica:** Precisará ser convertida para um tipo de data (`datetime`) para permitir análises temporais. Será a chave para nossa `d_calendario`.

#### Colunas Fato (As Métricas)
* **`Valor de Venda`**: Lido como texto (**`object`**). **Ação Crítica:** Precisa ser convertido para um tipo numérico (`DECIMAL`), tratando a vírgula como separador decimal.
* **`Valor de Compra`**: Lido como `object` (2019) ou `float64` (2025). **Problema de Qualidade Grave:** Esta coluna é muito pouco confiável. Tem muitos valores faltando em 2019 e está **completamente vazia** em 2025. **Decisão de Modelagem:** Devemos considerar se vale a pena incluir esta métrica, ou se devemos focarno `Valor de Venda` que é mais consistente.
* **`Unidade de Medida`**: Lido como texto (**`object`**). É uma dimensão que dá contexto aos fatos. Será importante para garantir que estamos comparando as mesmas unidades (ex: `R$ / litro`).

#### Padronização
* Os nomes das colunas contêm hífens, espaços e acentos. Todos precisarão ser padronizados para o formato `snake_case`.